In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re
import string

In [ ]:
pd.set_option('display.max_columns',600)
pd.set_option('display.max_rows',12000)

In [ ]:
Dframe=pd.read_csv('C:/Users/user/Desktop/IVY WORK BOOK/PYTHON/Python Datasets/Sentiment Analysis/superheroes_nlp_dataset.csv')

In [ ]:
Dframe.head(3)

In [ ]:
Dframe.info()

In [ ]:
Dframe1=pd.DataFrame(Dframe, columns=['history_text','powers_text','superpowers','aliases','alter_egos','creator'])

In [ ]:
Dframe1.tail(10)

In [ ]:
Dframe1.shape

In [ ]:
Dframe1.info()

In [ ]:
Dframe1=Dframe1.dropna()

In [ ]:
Dframe1['power of Text']=Dframe1['history_text']+' '+Dframe1['powers_text']+' '+Dframe1['superpowers']+' '+Dframe1['aliases']+' '+Dframe1['alter_egos']

In [ ]:
Dframe1=Dframe1.drop(labels=['history_text','powers_text','superpowers','aliases','alter_egos'], axis=1)

In [ ]:
Dframe1=Dframe1.dropna()

In [ ]:
Dframe1

In [ ]:
def cleaning1(inpdata):
    cleanedArticle1=re.sub(r'[?|$|(),"".@#=><|!]Â&*/',r' ',inpdata)
    cleanedArticle2=re.sub(r'[^a-z A-Z]',r' ',cleanedArticle1)
    cleanedArticle3=cleanedArticle2.lower()
    cleanedArticle4=re.sub(r'\b\w{1,3}\b', ' ',cleanedArticle3)
    cleanedArticle5=re.sub(r'https?://\S+|www\.\S+',r' ',cleanedArticle4)
    cleanedArticle6=re.sub(r' +', ' ',cleanedArticle5)
    return(cleanedArticle6)

In [ ]:
Dframe1['Pow of Text']=Dframe1['power of Text'].apply(cleaning1)

In [ ]:
def cleaning2(inpdata):
    cleanedArticle1=re.sub(r'[?|$|(),"".@#=><|!]Â&*/',r' ',inpdata)
    cleanedArticle2=re.sub(r'[^a-z A-Z]',r' ',cleanedArticle1)
    cleanedArticle3=cleanedArticle2.lower()
    cleanedArticle4=re.sub(r'\b\w{1,3}\b', ' ',cleanedArticle3)
    cleanedArticle5=re.sub(r'https?://\S+|www\.\S+',r' ',cleanedArticle4)
    cleanedArticle6=re.sub(r' +', ' ',cleanedArticle5)
    return(cleanedArticle6)

In [ ]:
Dframe1['creator1']=Dframe1['creator'].apply(cleaning2)

In [ ]:
Dframe1=Dframe1.drop(labels=['creator','power of Text'], axis=1)

In [ ]:
Dframe1.info()

In [ ]:
Dframe1=Dframe1.drop_duplicates('Pow of Text')

In [ ]:
Dframe1.groupby('creator1').count()

In [ ]:
Dframe1

In [ ]:
Dframe1.info()

In [ ]:
Dframe1['creater_New']=Dframe1[Dframe1['creator1'].str().split().len().lt(4)]

In [ ]:
Dframe1.info() 

In [ ]:
def Token_new1(inpdata):
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(inpdata)
    return(tokens)

In [ ]:
Dframe1['Pow_token']=Dframe1['Pow of Text'].apply(Token_new1)

In [ ]:
#def Token_new2(inpdata):
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(inpdata)
    return(tokens)

In [ ]:
#Dframe1['creator_token']=Dframe1['creator1'].apply(Token_new2)

In [ ]:
Dframe1=Dframe1.drop(labels=['Pow of Text'],axis=1)

In [ ]:
Dframe1.head()

In [ ]:
def remove(inpdata):
    return "".join(inpdata.split()) 

In [ ]:
Dframe1['creator_new']=Dframe1['creator1'].apply(remove)

In [ ]:
from sklearn.preprocessing import  LabelEncoder
le=LabelEncoder()

Dframe1['creator_Nw']=le.fit_transform(Dframe1['creator_new'])

In [ ]:
Dframe1=Dframe1.drop(labels=['creator1','creator_new'],axis=1)

In [ ]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 

In [ ]:
def Stem_Stop(Inpdata):
    texts=[]
    en_stop = get_stop_words('en')
    LM=WordNetLemmatizer()
    stopped_tokens = [i for i in Inpdata if not i in en_stop]
    porter_stem=[LM.lemmatize(word) for word in stopped_tokens ]
    texts.append(porter_stem)
    return(texts)

In [ ]:
Dframe1['Pow_token1']=Dframe1['Pow_token'].apply(Stem_Stop)

In [ ]:
Dframe1=Dframe1.drop(labels=['Pow_token'],axis=1)

In [ ]:
Dframe1_pkl=pd.to_pickle(Dframe1,'C:/Users/user/Desktop/IVY WORK BOOK/PYTHON/pickle files/Dframe1_pkl.pkl')

In [ ]:
Dframe1_pkl=pd.read_pickle('C:/Users/user/Desktop/IVY WORK BOOK/PYTHON/pickle files/Dframe1_pkl.pkl')

In [ ]:
import gensim 
from gensim.models import Word2Vec 

In [ ]:
corpus=Dframe1_pkl.Pow_token1
document=[''.join(str(text))for text in corpus]

In [ ]:
print(document[:10])

In [ ]:
Dframe1_pkl.head()

In [ ]:
Dframe1_pkl.info()


In [ ]:
from gensim.models import Word2Vec
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

In [ ]:
model=Word2Vec(document,size=150 ,min_count=2,alpha=0.005,window=10)

In [ ]:
print(model.wv.vocab)

In [ ]:
dframe2=pd.DataFrame(model.wv.vectors)

In [ ]:
dframe2.shape

In [ ]:
dframe2['Target']=Dframe1_pkl['creator_Nw']

In [ ]:
dframe2.info()

In [ ]:
dframe2['Target'].isnull().sum()

In [ ]:
mode=dframe2['Target'].mode()[0]

In [ ]:
dframe2['Target']=dframe2['Target'].fillna(mode)

In [ ]:
dframe2.groupby('Target').size().plot(kind='bar', figsize=(15,6))

In [ ]:
TArget=['Target']
y=dframe2[TArget].values
dframe2=dframe2.drop(labels=['Target'], axis=1)
Predictors=dframe2.columns
X=dframe2[Predictors].values

In [ ]:
y

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:

params={
 "learning_rate"    : [0.025] ,
 "max_depth"        : [15],
 "min_child_weight" : [5],
 "gamma"            : [0.1],
}

In [ ]:
from sklearn import metrics
metrics.SCORERS.keys()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier

In [ ]:
xgbclassifier=XGBClassifier()

In [ ]:
randomseacrch=RandomizedSearchCV(xgbclassifier,param_distributions=params,n_iter=7,scoring='f1_weighted',n_jobs=-1,cv=5,verbose=3)

In [ ]:
randomseacrch.fit(X,y)

In [ ]:
randomseacrch.best_params_

In [ ]:
xgbclassifier=XGBClassifier(min_child_weight= 5, max_depth= 15, learning_rate= 0.25, gamma=0.1,booster='gbtree',objective='binary:logistic')

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(xgbclassifier,X,y,cv=10)

In [ ]:
print(score)

In [ ]:
score.mean()

In [ ]:
from sklearn import svm

In [ ]:
params={'kernel':('rbf'), 'C':[1]}

In [ ]:
svc=svm.SVC()

In [ ]:
Gridsearch=GridSearchCV(svc,params,cv=5)

In [ ]:
Gridsearch.fit(X,y)

In [ ]:
Gridsearch.best_params_

In [ ]:
svc=svm.SVC(C=1.0,kernel='rbf',degree=3)

In [ ]:
score=cross_val_score(svc,X,y,cv=10)

In [ ]:
print(score)

In [ ]:
score.mean()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=750, test_size=0.32)

In [ ]:
xgbmodel=xgbclassifier.fit(X_train,y_train)
predictions=xgbmodel.predict(X_test)
print(metrics.classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))